# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [2]:
os.chdir('/home/workspace')
print(os.getcwd())

/home/workspace


#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Added manually to create empty list to hold os walk paths retrived
file_path_list = []

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list_append = glob.glob(os.path.join(root,'*'))
    # Added manually to append each os walk path to the created list
    file_path_list.append(file_path_list_append)
    # Watch out for the the ipynb file being added to the list as an additional list
    # To get only csv files access the first list at file_path_list[0]

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list[0]: # edited this line to file_path_list[0] based on previous cell edits 
# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# Create a Keyspace 

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS cassandra 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# Set KEYSPACE to the keyspace specified above

try:
    session.set_keyspace('cassandra')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

# Here I have created a Composite key of session_id and item_in_session as this will uniquely identify each row
# Regardless of artist or song data.
# As no sort order has been requested I have not included clustering columns

query = "CREATE TABLE IF NOT EXISTS artist_song_duration"
query = query + "(session_id int, item_in_session int, artist_name text, song_title text, song_length double, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [10]:
# Catch if mistake or inefficiency in data modelling of tables
# session.execute("DROP TABLE IF EXISTS artist_song_duration")

In [11]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO artist_song_duration (session_id, item_in_session, artist_name, song_title, song_length)"
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = "SELECT artist_name, song_title, song_length FROM artist_song_duration WHERE session_id = 338 AND item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.3073


The 4th item in session 338 was the song 'Music Matters (Mark Knight Dub)' by Faithless, and the song was 495.3073 seconds in length.

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [13]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

# Here I have created a Composite key of user_id and session_id with a clustering column of item_in_session
# When User_id and session_id are combined rows will become unique once the user and session
# Is specified in the WHERE clause of the SELECT query
# item_in_session will then order the rows in descending order as a clustering column

query = "CREATE TABLE IF NOT EXISTS artist_song_in_user_session"
query = query + "(user_id int, session_id int, item_in_session int, artist_name text, song_title text, user_first_name text, user_last_name text, PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [ ]:
# Catch if mistake or inefficiency in data modelling of tables
# session.execute("DROP TABLE IF EXISTS artist_song_in_user_session")

In [14]:
# Process file and insert records into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO artist_song_in_user_session (user_id, session_id, item_in_session, artist_name, song_title, user_first_name, user_last_name)"
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [15]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = "SELECT artist_name, song_title, item_in_session, user_first_name, user_last_name FROM artist_song_in_user_session WHERE user_id = 10 AND session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.item_in_session, row.user_first_name, row.user_last_name)

Down To The Bone Keep On Keepin' On 0 Sylvie Cruz
Three Drives Greece 2000 1 Sylvie Cruz
Sebastien Tellier Kilometer 2 Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 3 Sylvie Cruz


In this session user Sylvie Cruz listened to 4 songs: ('Keep on Keepin' On' : Down To The Bone), ('Greece 2000' : Three Drives), ('Kilometer' : Sebastien Tellier), and ('Catch You Baby (Steve Pitron & Max Sanna Radio Edit)' : Lonnie Gordon).

In [16]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# Here I have created a Composite key of song_title, user_first_name, user_last_name and user_id
# When combined as a primary key these columns will uniquely identify
# users who have listened to particular songs regardless of the number of times
# a song has been listened too either by total sessions (session_id) or within a session (item_in_session) 
# song_title must be included as either a partition key or clustering column to be used in the WHERE clause
# and user_first_name and user_last_name being included allows for a neat sort of the rows

query = "CREATE TABLE IF NOT EXISTS song_listened_by_user_Y_N"
query = query + "(song_title text, user_id int, user_first_name text, user_last_name text, PRIMARY KEY (song_title, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)        

In [17]:
# Catch if mistake or inefficiency in data modelling of tables
# session.execute("DROP TABLE IF EXISTS song_listened_by_user_Y_N")

In [18]:
# Process file and insert records into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_listened_by_user_Y_N (song_title, user_id, user_first_name, user_last_name)"
        query = query + "VALUES(%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [19]:
# Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "SELECT user_first_name, user_last_name FROM song_listened_by_user_Y_N WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_first_name, row.user_last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


Three users listened to the song 'All Hands Against His Own'. Jacqueline Lynch, Tegan Levine and Sara Johnson.

### Drop the tables before closing out the sessions

In [20]:
## Drop the table before closing out the sessions

query = "DROP TABLE artist_song_duration"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE artist_song_in_user_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE song_listened_by_user_Y_N"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()